Здесь я обрабатываю данные, скачанные в hw1p1.py

In [ ]:
!pip install nltk

In [ ]:
!pip install pymorphy2

In [10]:
!pip install sklearn

In [4]:
import nltk

In [5]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [6]:
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

In [7]:
from collections import Counter

In [8]:
from collections import OrderedDict

In [9]:
import json

In [11]:
from sklearn.metrics import accuracy_score

In [12]:
from pprint import pprint

In [2]:
with open('pos_rev.txt', 'r', encoding='utf-8') as p:
    pos_r = p.read()

In [1]:
with open('neg_rev.txt', 'r', encoding='utf-8') as n:
    neg_r = n.read()

### 2. *Токенизируйте слова, приведите их к нижнему регистру и к начальной форме (1 балл за токенизацию, 1 - за начальную форму)*


In [13]:
def collect_normform(text): # приводим слова к нижнему регистру и начальной форме
    words = []
    for word in nltk.word_tokenize(text.lower()):
        if word.isalpha():
            words.append(morph.parse(word)[0].normal_form)
    return words

In [16]:
n_normform = collect_normform(neg_r)

In [17]:
p_normform = collect_normform(pos_r)

### 3. Составьте 2 множества - в одном будут слова, которые встречаются **только** в положительных отзывах, а в другом - встречающиеся **только** в отрицательных. Попробуйте поиграть с частотностями и исключить шум (к примеру, выбросить слова, встречающиеся 1-2 раза) (3 балла) (если у вас получились пустые множества, уберите фильтр по частотности или увеличьте выборку)


In [64]:
def compare_lists(n_list, p_list, max_len=65): 
  #составляем множества со словами только из положительных и только из отрицательных отзывов
  freqlist_n = Counter()
  freqlist_p = Counter()
  for n in n_list:
    if n not in p_list:
      freqlist_n[n] += 1
  for p in p_list:
    if p not in n_list:
      freqlist_p[p] += 1
  return dict(freqlist_n.most_common(max_len)), dict(freqlist_p.most_common(max_len))

In [65]:
freqlist_n, freqlist_p = compare_lists(n_normform, p_normform)

In [66]:
freq_lists = {} #объединяем в один словарь
freq_lists['pos'] = freqlist_p
freq_lists['neg'] = freqlist_n

In [52]:
OrderedDict(freqlist_n)

OrderedDict([('мультфильм', 10),
             ('помощь', 8),
             ('попытаться', 7),
             ('касса', 6),
             ('пафосный', 6),
             ('матрица', 6),
             ('очевидный', 5),
             ('тяжёлый', 5),
             ('медицина', 5),
             ('непонятно', 5),
             ('парень', 5),
             ('даррелла', 5),
             ('награда', 5),
             ('косичка', 5),
             ('облик', 4),
             ('физический', 4),
             ('осознавать', 4),
             ('предназначить', 4),
             ('воля', 4),
             ('симпатия', 4),
             ('попытка', 4),
             ('никак', 4),
             ('влюбляться', 4),
             ('потратить', 4),
             ('способность', 4),
             ('спорить', 4),
             ('сотня', 4),
             ('преподносить', 4),
             ('основа', 4),
             ('костнера', 4),
             ('состояние', 4),
             ('отдать', 4),
             ('очевидно', 4),
             


### 4. Создайте функцию, которая будет определять, положительный ли отзыв или отрицательный в зависимости от того, какие слова встретились в нём, и посчитайте качество при помощи accuracy (1 - за коректно работающую функцию, 1 - за подсчёт accuracy)



In [27]:
test_reviews = json.load(open("test_r.json")) # открываем словарь с тестовыми текстами

In [ ]:
pprint(test_reviews)

In [37]:
def ton_detect(freq_lists, text): 
    counts = Counter()
    for tone, freq_list in freq_lists.items():
        freq_list = Counter(freq_list)
        for word in nltk.word_tokenize(text):
          morph.parse(word)[0].normal_form
          counts[tone] += int(freq_list[word] > 0)
    return counts.most_common()

In [67]:
for review, tone in test_reviews.items():
  counts = ton_detect(freq_lists, review)
  print(counts, tone)

[('pos', 0), ('neg', 0)] pos
[('pos', 3), ('neg', 2)] pos
[('pos', 5), ('neg', 3)] pos
[('pos', 2), ('neg', 1)] pos
[('pos', 1), ('neg', 1)] pos
[('neg', 5), ('pos', 0)] neg
[('pos', 1), ('neg', 0)] neg
[('neg', 2), ('pos', 1)] neg
[('pos', 1), ('neg', 1)] neg
[('neg', 2), ('pos', 0)] neg


### Считаем accuracy

In [30]:
def test_tone_detect(freq_lists, reviews_dict):
    results = [] 
    gold = []     
    
    for text, tone in reviews_dict.items():
      predicted_tone = ton_detect(freq_lists, text)[0][0]
      results.append(predicted_tone)
      gold.append(tone)
    print("Результаты:")
    print("Accuracy: %.4f" % accuracy_score(results, gold))

In [68]:
test_tone_detect(freq_lists, test_reviews)

RESULTS:
Test size: 5 texts per tone
Accuracy: 0.8000


Сначала accuracy была 0.6, и пишлось поиграть с длинной словарей, чтобы увеличить ее.

Как улучшить:
Во-первых, собрать больше отзывов. 

Во-вторых, можно составлять словарь из словосочетаний, а не из слов. 